##구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
IMAGE_width=46
IMAGE_height=56
IMAGE_size=(IMAGE_width,IMAGE_height)
IMAGE_channels=1

## 파일 경로 지정

In [ ]:
train_path='/gdrive/My Drive/생체인증보안/얼굴/train_set'
train_n_path='/gdrive/My Drive/생체인증보안/얼굴/train_set_n'
test_path='/gdrive/My Drive/생체인증보안/얼굴/test_set'

## 라이브러리 import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras import layers
from keras.preprocessing import image
from keras.models import Model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import tensorflow as tf
import random
import glob, os
import cv2
from PIL import Image

## Augmentation 후 저장

In [ ]:
# 랜덤시드 고정시키기
np.random.seed(15)

from keras.preprocessing.image import ImageDataGenerator, img_to_array

# 데이터셋 불러오기
data_aug_gen = ImageDataGenerator(brightness_range=[0.7,1.3],
                                  zoom_range=[0.9, 1.1],
                                  horizontal_flip=False,
                                  vertical_flip=False,
                                  fill_mode='constant')
 

In [ ]:
%cd '/gdrive/My Drive/생체인증보안/얼굴/train_set_n'
nowpath='/gdrive/My Drive/생체인증보안/얼굴/train_set_n'

for filename in os.listdir(nowpath):
  img=Image.open(filename)
  img = img_to_array(img)
  img = img.reshape((1,) + img.shape)
  
  i = 0
  # 이 for는 무한으로 반복되기 때문에 우리가 원하는 반복횟수를 지정하여, 지정된 반복횟수가 되면 빠져나오도록 해야합니다.
  for batch in data_aug_gen.flow(img, batch_size=1, save_to_dir='./', save_prefix=filename, save_format='BMP'):
     i += 1
     if i >9: 
         break

/gdrive/My Drive/생체인증보안/얼굴/train_set_n


In [ ]:
images=sorted(glob.glob("*"))
print(len(images))

11542


##데이터 불러오기

In [ ]:
x_data=[]
y_data=[]

file_names = os.listdir(train_n_path)
for file_name in file_names:
    file_path='{}/{}'.format(train_n_path, file_name)
    image=Image.open(file_path)
    image=image.resize((IMAGE_width,IMAGE_height))
    np_image=np.asarray(image)
    np_image=np_image/255

    name=file_name[0:4]
    y_name=int(name.lstrip("0"))
    x_data.append(np_image)
    y_data.append(y_name-1)


In [ ]:
x_data = np.array(x_data)
y_data = np.array(y_data)

In [ ]:
x_data.shape

(11542, 56, 46)

In [ ]:
x_data=x_data.reshape(11542,56,46,1)
x_data.shape

(11542, 56, 46, 1)

##모델링

In [ ]:
import tensorflow as tf
from keras import optimizers

model2 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, kernel_size=(3,3),strides=(1,1),
                         padding='same', activation='relu', 
                         input_shape=(IMAGE_height, IMAGE_width, IMAGE_channels)),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),
  tf.keras.layers.Dropout(0.25),

  tf.keras.layers.Conv2D(32, (3,3), activation='relu',padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Dropout(0.25),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(3000, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(350, activation='softmax')
])

adam=optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model2.compile(optimizer=adam, loss='sparse_categorical_crossentropy', 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
model2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 56, 46, 16)        160       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 28, 23, 16)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 28, 23, 16)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 28, 23, 32)        4640      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 14, 11, 32)        0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 14, 11, 32)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 4928)             

##Callback

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
#조기종료
earlystop=EarlyStopping(monitor='sparse_categorical_accuracy',patience=10)

#학습율 조정
learning_rate_reduction=ReduceLROnPlateau(monitor='sparse_categorical_accuracy',
                                          patience=2,
                                          verbose=1,
                                          factor=0.5,
                                          min_lr=0.000001)
#callback 설정
callback=[earlystop,learning_rate_reduction]

##모델 학습(Cross-Validation)

In [ ]:
from sklearn.model_selection import StratifiedKFold

kfold=StratifiedKFold(n_splits=5)
cvscores = []

for train_idx,val_idx in kfold.split(x_data,y_data):
  x_train,x_val=x_data[train_idx],x_data[val_idx]
  y_train,y_val=y_data[train_idx],y_data[val_idx]

  model2.fit(x_train,y_train,epochs=6,batch_size=10,verbose=1,callbacks=callback)
  
  scores = model2.evaluate(x_val, y_val, verbose=1)
  print("Acc: ",(scores[1]*100))
  cvscores.append(scores[1] * 100)

Epoch 1/6
924/924 [==============================] - 11s 11ms/step - loss: 5.6139 - sparse_categorical_accuracy: 0.0235
Epoch 2/6
924/924 [==============================] - 10s 11ms/step - loss: 2.3203 - sparse_categorical_accuracy: 0.4267
Epoch 3/6
924/924 [==============================] - 10s 11ms/step - loss: 0.8413 - sparse_categorical_accuracy: 0.7706
Epoch 4/6
924/924 [==============================] - 10s 11ms/step - loss: 0.3879 - sparse_categorical_accuracy: 0.8945
Epoch 5/6
924/924 [==============================] - 10s 11ms/step - loss: 0.2152 - sparse_categorical_accuracy: 0.9366
Epoch 6/6
73/73 [==============================] - 1s 5ms/step - loss: 0.0927 - sparse_categorical_accuracy: 0.9827
Acc:  98.26765060424805
Epoch 1/6
924/924 [==============================] - 10s 11ms/step - loss: 0.1096 - sparse_categorical_accuracy: 0.9687
Epoch 2/6
924/924 [==============================] - 10s 11ms/step - loss: 0.0758 - sparse_categorical_accuracy: 0.9765
Epoch 3/6
924/924 [=

In [ ]:
print("평균 검증 Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

평균 검증 Accuracy: 99.58% (+/- 0.66%)


In [ ]:
%cd '/gdrive/My Drive/생체인증보안/얼굴/'
model2.save('face_re.h5')

/gdrive/My Drive/생체인증보안/얼굴


##모델 성능 지표</br>
학습 데이터 중 split한 validation 데이터 사용

In [ ]:
y_train_pred=model2.predict(x_train)
y_train_pred

array([[5.9775292e-19, 1.1567196e-14, 6.6364406e-19, ..., 1.8832175e-19,
        6.4051244e-13, 3.9532251e-20],
       [9.3706503e-14, 1.7509484e-13, 2.6005811e-15, ..., 3.7666105e-13,
        1.4677293e-09, 3.4866995e-11],
       [1.3223259e-06, 2.1777332e-07, 2.4266167e-09, ..., 7.9478934e-07,
        3.0343056e-06, 3.0866538e-06],
       ...,
       [1.2387052e-18, 1.2849948e-14, 9.1755649e-15, ..., 1.1870763e-13,
        4.1340812e-14, 4.1816556e-14],
       [2.4009312e-12, 2.3219592e-12, 8.0016348e-11, ..., 2.6644495e-09,
        1.2154187e-10, 4.4639350e-09],
       [5.7585238e-15, 9.8096488e-13, 1.5654769e-12, ..., 9.2208963e-11,
        5.5130965e-12, 4.8166703e-11]], dtype=float32)

In [ ]:
print(y_train)

[234  83 265 ... 190 190 190]


In [ ]:
y_pred_train=[]
for i in range(len(y_train)):
  n=np.argmax(y_train_pred[i])
  y_pred_train.append(n)

In [ ]:
print(y_pred_train)

[234, 83, 265, 193, 280, 121, 314, 39, 268, 41, 328, 253, 105, 175, 285, 45, 279, 347, 328, 296, 27, 274, 212, 272, 265, 125, 107, 96, 47, 89, 65, 144, 65, 278, 209, 124, 131, 79, 316, 310, 0, 194, 290, 230, 86, 171, 60, 339, 185, 307, 302, 82, 105, 68, 269, 40, 18, 339, 22, 337, 128, 303, 195, 298, 198, 289, 67, 63, 94, 243, 171, 21, 187, 229, 19, 245, 287, 61, 134, 153, 282, 9, 342, 188, 160, 249, 90, 148, 132, 348, 55, 221, 138, 144, 293, 184, 253, 3, 192, 110, 228, 53, 99, 145, 232, 183, 308, 63, 86, 131, 80, 288, 165, 285, 95, 258, 196, 258, 17, 26, 143, 124, 299, 88, 117, 142, 182, 223, 176, 76, 206, 87, 213, 222, 117, 150, 5, 100, 3, 322, 157, 266, 177, 342, 61, 158, 292, 203, 181, 201, 246, 3, 273, 326, 146, 54, 122, 11, 81, 22, 327, 221, 254, 159, 198, 129, 326, 62, 85, 124, 196, 312, 245, 184, 140, 93, 8, 178, 173, 264, 245, 195, 315, 2, 181, 257, 313, 73, 169, 103, 143, 219, 40, 51, 278, 138, 235, 112, 40, 220, 64, 250, 66, 289, 80, 111, 132, 217, 68, 317, 133, 304, 261, 301

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(y_train,y_pred_train)                         
precision = precision_score(y_train,y_pred_train,average='micro') 
recall = recall_score(y_train,y_pred_train, average='micro')      
f1 = f1_score(y_train,y_pred_train, average='micro')

print('정확도: {0:.6f}, 정밀도: {1:.6f}, 재현율: {2:.6f}, F1-score: {3:.6f}'.
      format(accuracy, precision, recall, f1))

정확도: 1.000000, 정밀도: 1.000000, 재현율: 1.000000, F1-score: 1.000000


##테스트

In [ ]:
import natsort
%cd '/gdrive/My Drive/생체인증보안/얼굴/test_set'

fname=[]
x_test=[]
y_test=[]

for filename in os.listdir(test_path):
  fname.append(filename)

fname=natsort.natsorted(fname)

for filename in fname:
  img=Image.open(filename)
  print(filename)
  img=img.resize((IMAGE_width,IMAGE_height))
  np_image=np.asarray(img)
  np_image=np_image/256

  x_test.append(np_image)

x_test = np.array(x_test)

In [ ]:
x_test=x_test.reshape(700,56,46,1)
x_test.shape

(700, 56, 46, 1)

In [ ]:
np.save("/gdrive/My Drive/생체인증보안/얼굴/x_test.npy",x_test)

In [ ]:
x_test = np.load('/gdrive/My Drive/생체인증보안/얼굴/x_test.npy')

In [ ]:
y_pred=model2.predict(x_test)
print(y_pred)

[[1.1878618e-09 3.2184663e-01 5.9756655e-07 ... 3.2379086e-10
  8.3117845e-07 3.3264277e-07]
 [4.2216430e-09 2.7491516e-09 2.5983219e-08 ... 3.8335893e-05
  1.6859301e-06 2.7068916e-07]
 [3.5249578e-16 9.1653941e-16 3.7800982e-12 ... 5.5698951e-13
  4.9096829e-12 2.0778828e-13]
 ...
 [4.7585158e-20 4.7030652e-10 1.5217271e-18 ... 1.4168824e-08
  7.1888814e-16 2.5282805e-12]
 [3.3022530e-16 1.4715360e-05 3.2555059e-12 ... 1.8655376e-15
  7.3873909e-13 9.7351597e-15]
 [4.1784441e-18 6.7752547e-13 2.8821944e-17 ... 3.5382983e-12
  6.6111470e-11 9.1031351e-16]]


In [ ]:
result=[]
for i in range(700):
  n=np.argmax(y_pred[i])+1
  result.append(n)

In [ ]:
print(result)

[5, 270, 188, 219, 303, 256, 306, 47, 105, 153, 102, 266, 216, 177, 162, 82, 154, 32, 151, 53, 219, 247, 178, 227, 189, 317, 215, 145, 155, 260, 175, 282, 119, 284, 264, 309, 215, 67, 168, 234, 121, 179, 283, 290, 304, 166, 246, 97, 220, 184, 227, 133, 273, 172, 107, 131, 163, 54, 222, 225, 85, 347, 103, 155, 160, 98, 311, 200, 197, 258, 339, 203, 42, 345, 207, 333, 220, 149, 123, 231, 210, 91, 337, 261, 93, 219, 127, 48, 3, 309, 230, 157, 85, 128, 305, 6, 127, 215, 219, 118, 233, 132, 51, 326, 107, 51, 87, 110, 224, 29, 4, 227, 121, 160, 324, 53, 115, 194, 280, 233, 210, 188, 319, 98, 271, 213, 230, 57, 158, 287, 183, 49, 310, 177, 227, 319, 254, 337, 71, 342, 168, 241, 347, 220, 227, 164, 186, 96, 17, 101, 125, 296, 98, 192, 216, 227, 20, 154, 202, 219, 279, 128, 275, 20, 182, 70, 91, 187, 133, 220, 82, 126, 90, 144, 290, 224, 203, 294, 160, 296, 77, 254, 240, 149, 80, 212, 14, 199, 43, 321, 169, 278, 32, 116, 219, 39, 66, 167, 203, 217, 288, 212, 34, 216, 196, 29, 11, 53, 317, 77, 9

In [ ]:
col_name = ['Image', 'Answer']
list1 = [fname,result]
list_df = pd.DataFrame({'Image':fname,'Answer':result})
list_df.head(30)

In [ ]:
list_df.to_csv("/gdrive/My Drive/생체인증보안/얼굴/result_2차_2.csv")